In [4]:
import json
import datetime
import time
import requests
import nbimporter
import random
import os
import psutil
import psycopg2
from multiprocessing import Process, Pool
import Insert_py
import Get_py
import Tor
import Proxy_Gatherer as pg
import Init_database as indb

In [5]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [ ]:
#------- Gathering proxies for visiting teld.cn -------##
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
       'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
gatherproxy = pg.ProxyGatherer(hdr)
proxies_xici = gatherproxy.getproxies_xici()
proxies_elite = gatherproxy.getproxies_elite()
# t = time.time()
proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 
                                                                #format--(connect time, read time)
proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)

In [6]:
def write_error(error_log, pillcode, error_msg):
    """
        Input:
            error_log - file, write error messages into designated file
    """
    error_log.write(pillcode)
    error_log.write(str(error_msg))
    error_log.write('\n')

In [7]:
def pillState_scrap(pillcode): 
    """
        Input:
            pillCode - string, each charging pile has its own pile code
        Output:
            pill_detailedInfo - dictionary of pill state details
    """
    hdr = {
            'accept': "*/*",
            'user-agent': "Teld/3.3.0 (iPhone; iOS 10.3.3; Scale/2.00)",
            'cookie': "TELDAppID=",
            'content-length': "111",
            'connection': "keep-alive",
            'accept-encoding': "gzip, deflate",
            'device': "app_version=3.3.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
            'host': "basesg.teld.cn",
            'content-type': "application/x-www-form-urlencoded",
            'cache-control': "no-cache"
        }
    time_out = 200
    F = None
    proxies_pool = None
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
    except ValueError as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#             else:
            pill_detailedInfo = None
    except (requests.exceptions.ChunkedEncodingError) as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#         else:
            pill_detailedInfo = None
#     return pill_detailedInfo, proxies_pool
    return pill_detailedInfo

In [8]:
def insert_state(outputs):
    for pill_detailedInfo in outputs:
        if pill_detailedInfo is not None:
            db_name="db_teld"
            user_name="postgres"
            password = "postgres"
            host="127.0.0.1"
            port="5432"
            conn = indb.init_db(db_name, user_name, password, host, port)
            pillCode = str(pill_detailedInfo['pillCode'])
            stateCode = str(pill_detailedInfo['stateCode'])
            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
            if pill_detailedInfo['stateName'] is not None:
                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pill_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
            conn.close()

In [13]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request stationid which has user activities from db -------#
try:
    cur = conn.cursor()
    cur.execute("select distinct stationid from tbl_app_useractivity;")
    stationWithAct_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(stationWithAct_list))
# #------- Request pilecode from db for states -------#
staid_list = []
for staid in stationWithAct_list:
    staid_list.append(staid[0])
try:
    cur = conn.cursor()
    sta_list = ['dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66', '2b3a35d4-cb94-4b8f-a47a-bbb054754066']
    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',
    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7
    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',
    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(sta_list),))
    
#     stmt = "select pilecode from tbl_APP_pilecode where staid = 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'"
#     cur.execute(stmt)
#     cur.execute("select pilecode from tbl_APP_pilecode")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.

3128
20
8866


In [10]:
## ----- Test for single looping of multiprocessing post_request and insert -----##

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('Before runnning script, memory use:', memoryUse, 'MB')
# # loop_num = 1
# # max_proc =10000
# pool = Pool(200)
# t = time.time()
# if __name__ == '__main__':
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     print(len(results))
#     outputs = [p.get() for p in results]
# print("success, time costs is " + str(time.time()-t)) 

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('After runnning script,memory use:', memoryUse, 'MB')

In [17]:
for pillcode in pillcode_list:
    print(pillcode[0])

1101060331102
1101060331104
1101060331103
1101060331101
1101060331203
1101060331207
1101060331204
1101060331208
1101060331205
1101060331210
1101060331209
1101060331201
1101060331206
1101060331202
1101160216102
1101160216101
1101160216103
1101160216104
1101160216106
1101160216105


In [18]:
os.environ["https_proxy"]="https://165.84.167.54:8080"
os.environ["http_proxy"]="http://165.84.167.54:8080"

In [ ]:
# pool = Pool(processes=2)
endTime = datetime.datetime.now() + datetime.timedelta(hours=12)
hdr = {
        'accept': "*/*",
        'user-agent': "Teld/3.3.0 (iPhone; iOS 10.3.3; Scale/2.00)",
        'cookie': "TELDAppID=",
        'content-length': "111",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': "app_version=3.3.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
        }
while True:
    if datetime.datetime.now() >= endTime:
            break
#     Tor.renewTor()
#     Tor.connectTor()
#     Tor.showmyIp()
    proxy_idx = random.randint(0,len(proxiesElite_pool)-1)
    os.environ["https_proxy"]=proxiesElite_pool[proxy_idx]['https']
    os.environ["http_proxy"]=proxiesElite_pool[proxy_idx]['http']
    t = time.time()
    outputs=[]
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     outputs = [p.get() for p in results]
    for pillcode in pillcode_list:
#         Tor.connectTor()
        pill_detailedInfo = pillState_scrap(pillcode[0])
        outputs.append(pill_detailedInfo)
    insert_state(outputs)
    print("Takes "+str(time.time()-t)+" seconds")
    print("sleeping...")
    time.sleep(random.randint(600, 900))
    print("waking up!")

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
failed
Takes 102.696664095 seconds
sleeping...
waking up!
Takes 95.6789200306 seconds
sleeping...
waking up!
Takes 94.1043970585 seconds
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
failed
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
failed
HTTP

In [ ]:
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
pool = Pool(100)
t = time.time()
if __name__ == '__main__':
    results = [pool.apply_async(insert_state, (pill_detailedInfo,)) for pill_detailedInfo in outputs]
    print(len(results))
    outputs = [p.get() for p in results]
print("success, time costs is " + str(time.time()-t)) 
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script, memory use:', memoryUse, 'MB')